In [2]:
import scvi
import scanpy as sc
import numpy as np

/home/ceger/miniforge3/envs/pytorch-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ceger/miniforge3/envs/pytorch-env/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/ceger/miniforge3/envs/pytorch-env/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/ceger/miniforge3/envs/pytorch-env/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypath

In [35]:
import torch
from torch import nn
from torch.distributions import Normal
from torch.nn import ModuleList

In [4]:
adata = sc.read_h5ad(
    '../data/Marburg_cell_states_locked_scANVI_ctl230901.raw.h5ad'
)
adata

AnnData object with n_obs × n_vars = 97573 × 27208
    obs: 'sex', 'age', 'ethnicity', 'PaCO2', 'donor', 'infection', 'disease', 'SMK', 'illumina_stimunr', 'bd_rhapsody', 'n_genes', 'doublet_scores', 'predicted_doublets', 'batch', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'XIST-counts', 'S_score', 'G2M_score', 'condition', 'sample_group', 'IAV_score', 'group', 'Viral_score', 'cell_type', 'cell_states', 'leiden', 'cell_compartment', 'seed_labels', '_scvi_batch', '_scvi_labels', 'C_scANVI'
    var: 'mt', 'ribo'
    obsm: 'X_scANVI', 'X_scVI', 'X_umap'

In [5]:
adata.layers['counts'] = adata.X.copy()

In [6]:
sc.pp.highly_variable_genes(
    adata=adata,
    n_top_genes=3000,
    layer='counts',
    flavor='seurat_v3',
    batch_key='batch'
)

In [7]:
scvi.model.SCVI.setup_anndata(
    adata=adata,
    layer='counts',
    batch_key='donor',
)

In [8]:
model = scvi.model.SCVI(
    adata=adata,
    n_latent=50,
    n_hidden=3,
)

In [9]:
model

SCVI model with the following parameters: 
n_hidden: 3, n_latent: 50, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, 
latent_distribution: normal.
Training status: Not Trained
Model's adata is minified?: False

In [16]:
model._module_kwargs

{'n_hidden': 3,
 'n_latent': 50,
 'n_layers': 1,
 'dropout_rate': 0.1,
 'dispersion': 'gene',
 'gene_likelihood': 'zinb',
 'latent_distribution': 'normal'}

In [99]:
model.module

VAE(
  (z_encoder): Encoder(
    (encoder): FCLayers(
      (fc_layers): Sequential(
        (Layer 0): Sequential(
          (0): Linear(in_features=27208, out_features=3, bias=True)
          (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): None
          (3): ReLU()
          (4): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (mean_encoder): Linear(in_features=3, out_features=50, bias=True)
    (var_encoder): Linear(in_features=3, out_features=50, bias=True)
  )
  (l_encoder): Encoder(
    (encoder): FCLayers(
      (fc_layers): Sequential(
        (Layer 0): Sequential(
          (0): Linear(in_features=27208, out_features=3, bias=True)
          (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): None
          (3): ReLU()
          (4): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (mean_encoder): Linear(in_features=3, out_features=1, bias=True)
    (var_enco

## FC Layer Class

In [47]:
model.module.z_encoder.encoder.fc_layers

Sequential(
  (Layer 0): Sequential(
    (0): Linear(in_features=27208, out_features=3, bias=True)
    (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): None
    (3): ReLU()
    (4): Dropout(p=0.1, inplace=False)
  )
)

In [108]:
class FCLayers(nn.Module):

    def __init__(self, in_features, out_features, bias):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.bias = bias
        self.fc_layers = nn.Sequential(
            nn.Linear(
                in_features=in_features,
                out_features=out_features,
                bias=bias,
            ),
            nn.BatchNorm1d(
                3,
                eps=0.001,
                momentum=0.01,
                affine=True,
                track_running_stats=True
            ),
            nn.ReLU(),
            nn.Dropout(
                p=0.1,
                inplace=False,
            )

        )
    
    def forward(self, x):
        x = self.fc_layers(x)
        return x

In [109]:
FCLayers(
    in_features=27208,
    out_features=3,
    bias=True,
)

FCLayers(
  (fc_layers): Sequential(
    (0): Linear(in_features=27208, out_features=3, bias=True)
    (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
  )
)

## Encoder Class

In [74]:
model.module.z_encoder.encoder

FCLayers(
  (fc_layers): Sequential(
    (Layer 0): Sequential(
      (0): Linear(in_features=27208, out_features=3, bias=True)
      (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): None
      (3): ReLU()
      (4): Dropout(p=0.1, inplace=False)
    )
  )
)

In [76]:
class Encoder(nn.Module):

    def __init__(
            self,
            fc_in,
            fc_out,
            fc_bias,
    ):
        self.fc_in = fc_in
        self.fc_out = fc_out
        self.fc_bias = fc_bias
        super().__init__()
        self.encoder = FCLayers(
            in_features=fc_in,
            out_features=fc_out,
            bias=fc_bias,
        )

    def forward(self, x):
        pass

In [77]:
Encoder(
    fc_in=27208,
    fc_out=3,
    fc_bias=True
)

Encoder(
  (encoder): FCLayers(
    (fc_layers): Sequential(
      (0): Linear(in_features=27208, out_features=3, bias=True)
      (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.1, inplace=False)
    )
  )
)

## DecoderSCVI class

In [62]:
model.module.decoder

DecoderSCVI(
  (px_decoder): FCLayers(
    (fc_layers): Sequential(
      (Layer 0): Sequential(
        (0): Linear(in_features=62, out_features=3, bias=True)
        (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): None
        (3): ReLU()
        (4): None
      )
    )
  )
  (px_scale_decoder): Sequential(
    (0): Linear(in_features=3, out_features=27208, bias=True)
    (1): Softmax(dim=-1)
  )
  (px_r_decoder): Linear(in_features=3, out_features=27208, bias=True)
  (px_dropout_decoder): Linear(in_features=3, out_features=27208, bias=True)
)

In [93]:
class DecoderSCVI(nn.Module):

    def __init__(
            self,
            fc_in,
            fc_out,
            fc_bias,
            decoder_in,
            decoder_out,
            decoder_bias,
    ):
        super().__init__()
        self.fc_in = fc_in
        self.fc_out = fc_out
        self.fc_bias = fc_bias
        self.decoder_in = decoder_in
        self.decoder_out = decoder_out
        self.decoder_bias = decoder_bias
        self.px_decoder = FCLayers(
            in_features=fc_in,
            out_features=fc_out,
            bias=fc_bias,
        )
        self.px_scale_decoder = nn.Sequential(
            nn.Linear(
                in_features=decoder_in,
                out_features=decoder_out,
                bias=decoder_bias
            ),
            nn.Softmax(dim=1)
                    )
        self.px_r_decoder = nn.Linear(
            in_features=decoder_in,
            out_features=decoder_out,
            bias=decoder_bias
        )
        self.px_dropout_decoder = nn.Linear(
            in_features=decoder_in,
            out_features=decoder_out,
            bias=decoder_bias
        )

    def forward(self, x):
        pass

In [94]:
DecoderSCVI(
    fc_in=62,
    fc_out=3,
    fc_bias=True,
    decoder_in=3,
    decoder_out=27208,
    decoder_bias=True
)

DecoderSCVI(
  (px_decoder): FCLayers(
    (fc_layers): Sequential(
      (0): Linear(in_features=62, out_features=3, bias=True)
      (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.1, inplace=False)
    )
  )
  (px_scale_decoder): Sequential(
    (0): Linear(in_features=3, out_features=27208, bias=True)
    (1): Softmax(dim=1)
  )
  (px_r_decoder): Linear(in_features=3, out_features=27208, bias=True)
  (px_dropout_decoder): Linear(in_features=3, out_features=27208, bias=True)
)

## VAE Class

In [78]:
model.module

VAE(
  (z_encoder): Encoder(
    (encoder): FCLayers(
      (fc_layers): Sequential(
        (Layer 0): Sequential(
          (0): Linear(in_features=27208, out_features=3, bias=True)
          (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): None
          (3): ReLU()
          (4): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (mean_encoder): Linear(in_features=3, out_features=50, bias=True)
    (var_encoder): Linear(in_features=3, out_features=50, bias=True)
  )
  (l_encoder): Encoder(
    (encoder): FCLayers(
      (fc_layers): Sequential(
        (Layer 0): Sequential(
          (0): Linear(in_features=27208, out_features=3, bias=True)
          (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): None
          (3): ReLU()
          (4): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (mean_encoder): Linear(in_features=3, out_features=1, bias=True)
    (var_enco

In [97]:
class VAE(nn.Module):
    def __init__(
            self,
            z_enc_fc_in,
            z_enc_fc_out,
            z_enc_fc_bias,
            l_enc_fc_in,
            l_enc_fc_out,
            l_enc_fc_bias,
            decoder_fc_in,
            decoder_fc_out,
            decoder_fc_bias,
            decoder_in,
            decoder_out,
            decoder_bias,
    ):
        self.z_enc_fc_in = z_enc_fc_in
        self.z_enc_fc_out = z_enc_fc_out
        self.z_enc_fc_bias = z_enc_fc_bias
        self.l_enc_fc_in = l_enc_fc_in
        self.l_enc_fc_out = l_enc_fc_out
        self.l_enc_fc_bias = l_enc_fc_bias
        self.decoder_fc_in = decoder_fc_in
        self.decoder_fc_out = decoder_fc_out
        self.decoder_fc_bias = decoder_fc_bias
        self.decoder_in = decoder_in
        self.decoder_out = decoder_out
        self.decoder_bias = decoder_bias
        super().__init__()
        self.z_encoder = Encoder(
            fc_in=z_enc_fc_in,
            fc_out=z_enc_fc_out,
            fc_bias=z_enc_fc_bias,
        )
        self.l_encoder = Encoder(
            fc_in=l_enc_fc_in,
            fc_out=l_enc_fc_out,
            fc_bias=l_enc_fc_bias,
        )
        self.decoder = DecoderSCVI(
            fc_in=decoder_fc_in,
            fc_out=decoder_fc_out,
            fc_bias=decoder_fc_bias,
            decoder_in=decoder_in,
            decoder_out=decoder_out,
            decoder_bias=decoder_out
        )

    def forward(self, x):
        pass


In [107]:
VAE(
    z_enc_fc_in=27208,
    z_enc_fc_out=3,
    z_enc_fc_bias=True,
    l_enc_fc_in=27208,
    l_enc_fc_out=3,
    l_enc_fc_bias=True,
    decoder_fc_in=62,
    decoder_fc_out=3,
    decoder_fc_bias=True,
    decoder_in=3,
    decoder_out=27208,
    decoder_bias=True,
).z_encoder.encoder.fc_layers

Sequential(
  (0): Linear(in_features=27208, out_features=3, bias=True)
  (1): BatchNorm1d(3, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.1, inplace=False)
)